# 제조 AI 실습 과정 - Session 3 (4-6시간)
## 예지보전(시계열) 심화

### 학습 목표
- 윈도우별 FFT 주요 통계피처(RMS/Kurtosis) 추출 및 분석
- Isolation Forest/OCSVM vs LSTM/AE 비교 실습
- 경보 정책 알람 피로도 스트리밍 파이프라인 구축

---

한글 깨짐 방지

Step 1: 실행

In [ ]:
!apt-get install -y fonts-nanum*
!rm -rf /root/.cache/matplotlib/* # 폰트 캐시 재설정
print("=" * 50)
print("✅ 폰트 설치 완료!")
print("⚠️ 이제 다음을 수행하세요:")
print("1. 상단 메뉴: 런타임 → 런타임 다시 시작")
print("2. 재시작 후 Step 2 코드 실행")
print("=" * 50)

Step 2: 실행

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
#
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
plt.rcParams['font.family'] = font_name

# 확인
plt.plot([1,2,3], [1,2,3])
plt.title('한글 테스트 성공! 🎉')
plt.xlabel('한글 X축')
plt.ylabel('한글 Y축')
plt.show()

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# 필요 라이브러리 설치
!pip install scipy numpy pandas matplotlib seaborn -q
!pip install scikit-learn -q
!pip install tensorflow -q
!pip install pywavelets -q  # Wavelet Transform
!pip install statsmodels -q
!pip install pyod -q  # Outlier Detection
!pip install streamz -q  # Streaming data processing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Signal Processing
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.stats import kurtosis, skew
import pywt

# Machine Learning
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_style('whitegrid')

# Check GPU
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print("✅ 라이브러리 로드 완료")

## 2. 설비 시계열 데이터 시뮬레이션

### 2.1 베어링 진동 데이터 생성

In [ ]:
class BearingVibrationSimulator:
    """
    베어링 진동 데이터 시뮬레이터
    정상 → 초기 결함 → 진행된 결함 → 고장 단계를 시뮬레이션
    """
    def __init__(self, sampling_rate=10000):  # 10 kHz
        self.sampling_rate = sampling_rate
        self.bearing_frequencies = {
            'BPFO': 236.4,  # Ball Pass Frequency Outer race
            'BPFI': 296.8,  # Ball Pass Frequency Inner race
            'BSF': 139.9,   # Ball Spin Frequency
            'FTF': 14.8     # Fundamental Train Frequency
        }

    def generate_normal_vibration(self, duration=10):
        """정상 진동 신호 생성"""
        t = np.linspace(0, duration, int(self.sampling_rate * duration))

        # 기본 회전 주파수 (30 Hz = 1800 RPM)
        rotation_freq = 30

        # 정상 신호: 회전 주파수 + 랜덤 노이즈
        signal = (
            0.5 * np.sin(2 * np.pi * rotation_freq * t) +  # 1X
            0.2 * np.sin(2 * np.pi * 2 * rotation_freq * t) +  # 2X
            0.1 * np.sin(2 * np.pi * 3 * rotation_freq * t) +  # 3X
            0.3 * np.random.normal(0, 0.1, len(t))  # 노이즈
        )

        return t, signal

    def generate_faulty_vibration(self, duration=10, fault_type='outer_race', severity='early'):
        """결함 진동 신호 생성"""
        t, normal_signal = self.generate_normal_vibration(duration)

        # 결함 주파수 선택
        if fault_type == 'outer_race':
            fault_freq = self.bearing_frequencies['BPFO']
        elif fault_type == 'inner_race':
            fault_freq = self.bearing_frequencies['BPFI']
        elif fault_type == 'ball':
            fault_freq = self.bearing_frequencies['BSF']
        else:
            fault_freq = self.bearing_frequencies['FTF']

        # 결함 심각도에 따른 진폭 설정
        severity_amplitude = {
            'early': 0.3,
            'moderate': 0.8,
            'severe': 2.0
        }

        amp = severity_amplitude.get(severity, 0.5)

        # 충격 신호 생성 (결함 특성)
        impact_interval = 1 / fault_freq
        impacts = np.zeros_like(t)

        for i in range(int(duration / impact_interval)):
            impact_time = i * impact_interval
            impact_idx = int(impact_time * self.sampling_rate)
            if impact_idx < len(impacts):
                # 지수 감쇠 충격 신호
                decay_length = min(100, len(impacts) - impact_idx)
                impacts[impact_idx:impact_idx+decay_length] += (
                    amp * np.exp(-np.arange(decay_length) * 0.1) *
                    np.sin(2 * np.pi * 2000 * np.arange(decay_length) / self.sampling_rate)
                )

        # 결함 신호 = 정상 신호 + 충격 신호
        faulty_signal = normal_signal + impacts

        # 결함이 진행될수록 노이즈 증가
        if severity == 'severe':
            faulty_signal += np.random.normal(0, 0.3, len(t))

        return t, faulty_signal

    def generate_degradation_sequence(self, total_days=30, samples_per_day=10):
        """설비 열화 과정 시뮬레이션"""
        degradation_data = []
        labels = []
        timestamps = []

        for day in range(total_days):
            # 열화 단계 결정
            if day < total_days * 0.3:
                stage = 'normal'
                label = 0
            elif day < total_days * 0.6:
                stage = 'early'
                label = 1
            elif day < total_days * 0.85:
                stage = 'moderate'
                label = 2
            else:
                stage = 'severe'
                label = 3

            for sample in range(samples_per_day):
                timestamp = datetime.now() + timedelta(days=day, hours=sample*2)

                if stage == 'normal':
                    _, signal = self.generate_normal_vibration(duration=1)
                else:
                    _, signal = self.generate_faulty_vibration(
                        duration=1,
                        fault_type='outer_race',
                        severity=stage
                    )

                degradation_data.append(signal)
                labels.append(label)
                timestamps.append(timestamp)

        return np.array(degradation_data), np.array(labels), timestamps

# 베어링 진동 데이터 생성
simulator = BearingVibrationSimulator()

# 정상 vs 결함 신호 비교
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
fig.suptitle('베어링 진동 신호: 정상 vs 결함 단계', fontsize=16, fontweight='bold')

# 시간 도메인 신호
stages = [('normal', None), ('early', 'early'), ('severe', 'severe')]

for idx, (ax_time, ax_freq) in enumerate(axes):
    stage_name, severity = stages[idx]

    if stage_name == 'normal':
        t, signal = simulator.generate_normal_vibration(duration=0.5)
        title = 'Normal Operation'
    else:
        t, signal = simulator.generate_faulty_vibration(duration=0.5, severity=severity)
        title = f'Fault: {severity.capitalize()} Stage'

    # Time domain
    ax_time.plot(t[:5000], signal[:5000], linewidth=0.5)
    ax_time.set_title(f'{title} - Time Domain')
    ax_time.set_xlabel('Time (s)')
    ax_time.set_ylabel('Amplitude (g)')
    ax_time.grid(True, alpha=0.3)

    # Frequency domain (FFT)
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1/simulator.sampling_rate)[:N//2]

    ax_freq.semilogy(xf[:2000], 2.0/N * np.abs(yf[:2000]))
    ax_freq.set_title(f'{title} - Frequency Domain')
    ax_freq.set_xlabel('Frequency (Hz)')
    ax_freq.set_ylabel('Magnitude')
    ax_freq.grid(True, alpha=0.3)

    # 결함 주파수 표시
    if stage_name != 'normal':
        ax_freq.axvline(x=simulator.bearing_frequencies['BPFO'],
                       color='r', linestyle='--', alpha=0.5, label='BPFO')
        ax_freq.legend()

plt.tight_layout()
plt.show()

print("✅ 베어링 진동 시뮬레이션 완료")
print(f"   - Sampling Rate: {simulator.sampling_rate} Hz")
print(f"   - Bearing Fault Frequencies:")
for fault, freq in simulator.bearing_frequencies.items():
    print(f"     {fault}: {freq:.1f} Hz")

## 3. 특징 추출 (Feature Engineering)

### 3.1 시간 도메인 특징 추출

In [ ]:
class TimeFeatureExtractor:
    """시간 도메인 특징 추출기"""

    @staticmethod
    def extract_statistical_features(signal):
        """통계적 특징 추출"""
        features = {}

        # 기본 통계량
        features['mean'] = np.mean(signal)
        features['std'] = np.std(signal)
        features['max'] = np.max(signal)
        features['min'] = np.min(signal)
        features['rms'] = np.sqrt(np.mean(signal**2))  # Root Mean Square

        # 고차 통계량
        features['skewness'] = skew(signal)
        features['kurtosis'] = kurtosis(signal)  # 충격 신호 감지에 유용

        # Peak 관련 특징
        features['peak'] = np.max(np.abs(signal))
        features['peak_to_peak'] = features['max'] - features['min']
        features['crest_factor'] = features['peak'] / features['rms'] if features['rms'] != 0 else 0

        # Shape Factor
        features['shape_factor'] = features['rms'] / np.mean(np.abs(signal)) if np.mean(np.abs(signal)) != 0 else 0

        # Impulse Factor
        features['impulse_factor'] = features['peak'] / np.mean(np.abs(signal)) if np.mean(np.abs(signal)) != 0 else 0

        # Clearance Factor
        features['clearance_factor'] = features['peak'] / (np.mean(np.sqrt(np.abs(signal)))**2) if np.mean(np.sqrt(np.abs(signal))) != 0 else 0

        return features

    @staticmethod
    def extract_waveform_features(signal):
        """파형 특징 추출"""
        features = {}

        # Zero Crossing Rate
        zero_crossings = np.where(np.diff(np.signbit(signal)))[0]
        features['zero_crossing_rate'] = len(zero_crossings) / len(signal)

        # Mean Absolute Deviation
        features['mad'] = np.mean(np.abs(signal - np.mean(signal)))

        # Energy
        features['energy'] = np.sum(signal**2)

        # Shannon Entropy
        hist, _ = np.histogram(signal, bins=50)
        hist = hist / hist.sum()
        hist = hist[hist > 0]  # Remove zeros
        features['entropy'] = -np.sum(hist * np.log2(hist))

        return features

# 시간 도메인 특징 추출 테스트
time_extractor = TimeFeatureExtractor()

# 정상 신호와 결함 신호 비교
_, normal_signal = simulator.generate_normal_vibration(duration=1)
_, faulty_signal = simulator.generate_faulty_vibration(duration=1, severity='severe')

normal_features = time_extractor.extract_statistical_features(normal_signal)
faulty_features = time_extractor.extract_statistical_features(faulty_signal)

# 특징 비교 시각화
feature_comparison = pd.DataFrame({
    'Normal': normal_features,
    'Faulty': faulty_features
})

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Feature comparison bar plot
feature_comparison.T.plot(kind='bar', ax=axes[0])
axes[0].set_title('Time Domain Features: Normal vs Faulty')
axes[0].set_xlabel('Condition')
axes[0].set_ylabel('Feature Value')
axes[0].legend(loc='upper left', bbox_to_anchor=(1, 1))
axes[0].grid(True, alpha=0.3)

# Feature importance (relative change)
relative_change = ((feature_comparison['Faulty'] - feature_comparison['Normal']) /
                  feature_comparison['Normal'].abs() * 100)
relative_change = relative_change.sort_values(ascending=True)

axes[1].barh(range(len(relative_change)), relative_change.values)
axes[1].set_yticks(range(len(relative_change)))
axes[1].set_yticklabels(relative_change.index)
axes[1].set_xlabel('Relative Change (%)')
axes[1].set_title('Feature Sensitivity to Fault')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 시간 도메인 특징 비교:")
print(feature_comparison.round(3))

### 3.2 주파수 도메인 특징 추출 (FFT 기반)

In [ ]:
class FrequencyFeatureExtractor:
    """주파수 도메인 특징 추출기"""

    def __init__(self, sampling_rate):
        self.sampling_rate = sampling_rate

    def extract_fft_features(self, signal, n_bands=10):
        """FFT 기반 특징 추출"""
        features = {}

        # FFT 계산
        N = len(signal)
        yf = fft(signal)
        xf = fftfreq(N, 1/self.sampling_rate)[:N//2]
        magnitude = 2.0/N * np.abs(yf[:N//2])

        # 주파수 대역 나누기
        freq_bands = np.linspace(0, self.sampling_rate//2, n_bands+1)

        # 각 대역의 에너지 계산
        for i in range(n_bands):
            band_mask = (xf >= freq_bands[i]) & (xf < freq_bands[i+1])
            features[f'band_{i}_energy'] = np.sum(magnitude[band_mask]**2)

        # 전체 스펙트럼 특징
        features['total_power'] = np.sum(magnitude**2)
        features['mean_frequency'] = np.sum(xf * magnitude) / np.sum(magnitude) if np.sum(magnitude) > 0 else 0

        # Peak frequency
        peak_idx = np.argmax(magnitude)
        features['peak_frequency'] = xf[peak_idx]
        features['peak_magnitude'] = magnitude[peak_idx]

        # Spectral entropy
        psd = magnitude**2 / np.sum(magnitude**2)
        psd = psd[psd > 0]
        features['spectral_entropy'] = -np.sum(psd * np.log2(psd))

        # Spectral centroid (무게중심)
        features['spectral_centroid'] = np.sum(xf * magnitude) / np.sum(magnitude) if np.sum(magnitude) > 0 else 0

        # Spectral spread
        if features['spectral_centroid'] > 0:
            features['spectral_spread'] = np.sqrt(
                np.sum(((xf - features['spectral_centroid'])**2) * magnitude) / np.sum(magnitude)
            )
        else:
            features['spectral_spread'] = 0

        return features, xf, magnitude

    def extract_envelope_spectrum(self, signal):
        """Envelope spectrum 분석 (베어링 결함 감지에 효과적)"""
        # Hilbert transform으로 envelope 추출
        analytic_signal = signal.hilbert(signal)
        envelope = np.abs(analytic_signal)

        # Envelope의 FFT
        N = len(envelope)
        yf = fft(envelope)
        xf = fftfreq(N, 1/self.sampling_rate)[:N//2]
        magnitude = 2.0/N * np.abs(yf[:N//2])

        return xf, magnitude

# 주파수 특징 추출
freq_extractor = FrequencyFeatureExtractor(simulator.sampling_rate)

# Window별 FFT 특징 추출
def extract_windowed_features(signal, window_size=1024, overlap=0.5):
    """윈도우 단위로 특징 추출"""
    step_size = int(window_size * (1 - overlap))
    n_windows = (len(signal) - window_size) // step_size + 1

    all_features = []

    for i in range(n_windows):
        start = i * step_size
        end = start + window_size
        window = signal[start:end]

        # 시간 도메인 특징
        time_features = time_extractor.extract_statistical_features(window)

        # 주파수 도메인 특징
        freq_features, _, _ = freq_extractor.extract_fft_features(window)

        # 특징 결합
        combined_features = {**time_features, **freq_features}
        all_features.append(combined_features)

    return pd.DataFrame(all_features)

# 열화 시뮬레이션 데이터에서 특징 추출
degradation_signals, degradation_labels, timestamps = simulator.generate_degradation_sequence(
    total_days=10, samples_per_day=5
)

# 각 신호에서 특징 추출
all_extracted_features = []
for signal in degradation_signals:
    features_df = extract_windowed_features(signal)
    all_extracted_features.append(features_df.mean())  # 평균 특징 사용

features_matrix = pd.DataFrame(all_extracted_features)
features_matrix['label'] = degradation_labels
features_matrix['timestamp'] = timestamps

# 주요 특징 변화 시각화
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('Feature Evolution During Equipment Degradation', fontsize=16, fontweight='bold')

key_features = ['rms', 'kurtosis', 'crest_factor', 'peak_frequency', 'spectral_entropy', 'total_power']

for ax, feature in zip(axes.flat, key_features):
    if feature in features_matrix.columns:
        ax.plot(range(len(features_matrix)), features_matrix[feature].values)

        # 상태별 색상 구분
        colors = ['green', 'yellow', 'orange', 'red']
        for label in range(4):
            mask = features_matrix['label'] == label
            indices = np.where(mask)[0]
            if len(indices) > 0:
                ax.scatter(indices, features_matrix[feature].values[mask],
                          c=colors[label], alpha=0.6, s=20)

        ax.set_title(f'{feature.replace("_", " ").title()}')
        ax.set_xlabel('Sample Index')
        ax.set_ylabel('Feature Value')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✅ 특징 추출 완료")
print(f"   - 총 샘플 수: {len(features_matrix)}")
print(f"   - 추출된 특징 수: {len(features_matrix.columns) - 2}")  # label, timestamp 제외
print(f"\n상태별 샘플 분포:")
print(features_matrix['label'].value_counts().sort_index())

## 4. 이상 탐지 모델 비교

### 4.1 Isolation Forest vs One-Class SVM

In [ ]:
class AnomalyDetectionComparison:
    """이상 탐지 모델 비교 클래스"""

    def __init__(self, features_df, labels):
        self.features = features_df.drop(['label', 'timestamp'], axis=1, errors='ignore')
        self.labels = labels
        self.scaler = StandardScaler()
        self.features_scaled = self.scaler.fit_transform(self.features)

    def train_isolation_forest(self, contamination=0.1):
        """Isolation Forest 학습"""
        model = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_estimators=100
        )

        # 정상 데이터로만 학습 (label=0)
        normal_data = self.features_scaled[self.labels == 0]
        model.fit(normal_data)

        # 전체 데이터 예측
        predictions = model.predict(self.features_scaled)
        scores = model.score_samples(self.features_scaled)

        return model, predictions, scores

    def train_ocsvm(self, nu=0.1, kernel='rbf'):
        """One-Class SVM 학습"""
        model = OneClassSVM(
            nu=nu,
            kernel=kernel,
            gamma='auto'
        )

        # 정상 데이터로만 학습
        normal_data = self.features_scaled[self.labels == 0]
        model.fit(normal_data)

        # 전체 데이터 예측
        predictions = model.predict(self.features_scaled)
        scores = model.decision_function(self.features_scaled)

        return model, predictions, scores

    def evaluate_model(self, predictions, model_name):
        """모델 성능 평가"""
        # 이진 분류로 변환 (0: 정상, 1-3: 이상)
        binary_labels = (self.labels > 0).astype(int)
        binary_predictions = (predictions == -1).astype(int)  # -1: 이상, 1: 정상

        # 성능 지표 계산
        cm = confusion_matrix(binary_labels, binary_predictions)
        report = classification_report(binary_labels, binary_predictions,
                                      target_names=['Normal', 'Anomaly'])

        print(f"\n{'='*50}")
        print(f"{model_name} Performance")
        print(f"{'='*50}")
        print("\nConfusion Matrix:")
        print(cm)
        print("\nClassification Report:")
        print(report)

        return cm

    def compare_models(self):
        """모델 비교 시각화"""
        # 모델 학습
        if_model, if_pred, if_scores = self.train_isolation_forest()
        ocsvm_model, ocsvm_pred, ocsvm_scores = self.train_ocsvm()

        # 평가
        if_cm = self.evaluate_model(if_pred, "Isolation Forest")
        ocsvm_cm = self.evaluate_model(ocsvm_pred, "One-Class SVM")

        # 시각화
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('Anomaly Detection Model Comparison', fontsize=16, fontweight='bold')

        # Isolation Forest Results
        axes[0, 0].plot(if_scores)
        axes[0, 0].scatter(range(len(if_scores)), if_scores,
                          c=['red' if x == -1 else 'blue' for x in if_pred],
                          alpha=0.5, s=10)
        axes[0, 0].set_title('Isolation Forest - Anomaly Scores')
        axes[0, 0].set_xlabel('Sample Index')
        axes[0, 0].set_ylabel('Anomaly Score')
        axes[0, 0].grid(True, alpha=0.3)

        # OCSVM Results
        axes[1, 0].plot(ocsvm_scores)
        axes[1, 0].scatter(range(len(ocsvm_scores)), ocsvm_scores,
                          c=['red' if x == -1 else 'blue' for x in ocsvm_pred],
                          alpha=0.5, s=10)
        axes[1, 0].set_title('One-Class SVM - Decision Scores')
        axes[1, 0].set_xlabel('Sample Index')
        axes[1, 0].set_ylabel('Decision Score')
        axes[1, 0].grid(True, alpha=0.3)

        # Confusion Matrices
        sns.heatmap(if_cm, annot=True, fmt='d', ax=axes[0, 1], cmap='Blues')
        axes[0, 1].set_title('Isolation Forest - Confusion Matrix')
        axes[0, 1].set_xlabel('Predicted')
        axes[0, 1].set_ylabel('Actual')

        sns.heatmap(ocsvm_cm, annot=True, fmt='d', ax=axes[1, 1], cmap='Oranges')
        axes[1, 1].set_title('One-Class SVM - Confusion Matrix')
        axes[1, 1].set_xlabel('Predicted')
        axes[1, 1].set_ylabel('Actual')

        # Detection over time
        axes[0, 2].plot(self.labels, 'g-', label='True Labels', alpha=0.7)
        axes[0, 2].scatter(range(len(if_pred)),
                          [3 if x == -1 else 0 for x in if_pred],
                          c='red', alpha=0.5, s=10, label='IF Detections')
        axes[0, 2].set_title('Isolation Forest - Detection Timeline')
        axes[0, 2].set_xlabel('Time')
        axes[0, 2].set_ylabel('State')
        axes[0, 2].legend()
        axes[0, 2].grid(True, alpha=0.3)

        axes[1, 2].plot(self.labels, 'g-', label='True Labels', alpha=0.7)
        axes[1, 2].scatter(range(len(ocsvm_pred)),
                          [3 if x == -1 else 0 for x in ocsvm_pred],
                          c='orange', alpha=0.5, s=10, label='OCSVM Detections')
        axes[1, 2].set_title('One-Class SVM - Detection Timeline')
        axes[1, 2].set_xlabel('Time')
        axes[1, 2].set_ylabel('State')
        axes[1, 2].legend()
        axes[1, 2].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

        return if_model, ocsvm_model

# 이상 탐지 모델 비교 실행
anomaly_detector = AnomalyDetectionComparison(features_matrix, degradation_labels)
if_model, ocsvm_model = anomaly_detector.compare_models()

### 4.2 LSTM Autoencoder for Anomaly Detection

In [ ]:
class LSTMAutoencoder:
    """LSTM 기반 Autoencoder for 시계열 이상 탐지"""

    def __init__(self, sequence_length=50, n_features=10):
        self.sequence_length = sequence_length
        self.n_features = n_features
        self.model = None
        self.threshold = None

    def build_model(self):
        """LSTM Autoencoder 모델 구성"""
        # Encoder
        inputs = keras.Input(shape=(self.sequence_length, self.n_features))

        # Encoder layers
        encoded = layers.LSTM(128, activation='relu', return_sequences=True)(inputs)
        encoded = layers.Dropout(0.2)(encoded)
        encoded = layers.LSTM(64, activation='relu', return_sequences=True)(encoded)
        encoded = layers.Dropout(0.2)(encoded)
        encoded = layers.LSTM(32, activation='relu', return_sequences=False)(encoded)

        # Bottleneck
        encoded = layers.RepeatVector(self.sequence_length)(encoded)

        # Decoder layers
        decoded = layers.LSTM(32, activation='relu', return_sequences=True)(encoded)
        decoded = layers.Dropout(0.2)(decoded)
        decoded = layers.LSTM(64, activation='relu', return_sequences=True)(decoded)
        decoded = layers.Dropout(0.2)(decoded)
        decoded = layers.LSTM(128, activation='relu', return_sequences=True)(decoded)

        # Output layer
        outputs = layers.TimeDistributed(layers.Dense(self.n_features))(decoded)

        # Model
        self.model = keras.Model(inputs=inputs, outputs=outputs)
        self.model.compile(optimizer='adam', loss='mse')

        return self.model

    def prepare_sequences(self, data, labels=None):
        """시퀀스 데이터 준비"""
        sequences = []
        sequence_labels = [] if labels is not None else None

        for i in range(len(data) - self.sequence_length + 1):
            seq = data[i:i+self.sequence_length]
            sequences.append(seq)

            if labels is not None:
                # 시퀀스의 마지막 라벨 사용
                sequence_labels.append(labels[i+self.sequence_length-1])

        return np.array(sequences), np.array(sequence_labels) if labels is not None else None

    def train(self, normal_data, epochs=50, batch_size=32):
        """정상 데이터로 학습"""
        # 모델 빌드
        if self.model is None:
            self.build_model()

        # 학습
        history = self.model.fit(
            normal_data, normal_data,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.1,
            verbose=0
        )

        # 정상 데이터의 재구성 오차 계산
        predictions = self.model.predict(normal_data, verbose=0)
        mse = np.mean(np.square(normal_data - predictions), axis=(1, 2))

        # 임계값 설정 (평균 + 3*표준편차)
        self.threshold = np.mean(mse) + 3 * np.std(mse)

        return history

    def detect_anomalies(self, data):
        """이상 탐지"""
        predictions = self.model.predict(data, verbose=0)
        mse = np.mean(np.square(data - predictions), axis=(1, 2))

        # 이상 판정
        anomalies = mse > self.threshold

        return anomalies, mse

# LSTM Autoencoder 학습 및 테스트
# 특징 데이터 준비
features_for_lstm = features_matrix.drop(['label', 'timestamp'], axis=1, errors='ignore').values
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_for_lstm)

# 시퀀스 생성
sequence_length = 10
lstm_ae = LSTMAutoencoder(sequence_length=sequence_length, n_features=features_scaled.shape[1])
sequences, sequence_labels = lstm_ae.prepare_sequences(features_scaled, degradation_labels)

# 정상 데이터만 선택 (label=0)
normal_sequences = sequences[sequence_labels == 0]

# 모델 학습
print("🚀 LSTM Autoencoder 학습 중...")
history = lstm_ae.train(normal_sequences, epochs=30, batch_size=16)

# 전체 데이터에 대한 이상 탐지
anomalies, reconstruction_errors = lstm_ae.detect_anomalies(sequences)

# 결과 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('LSTM Autoencoder Anomaly Detection', fontsize=16, fontweight='bold')

# Training history
axes[0, 0].plot(history.history['loss'], label='Training Loss')
axes[0, 0].plot(history.history['val_loss'], label='Validation Loss')
axes[0, 0].set_title('Model Training History')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss (MSE)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Reconstruction errors
axes[0, 1].plot(reconstruction_errors)
axes[0, 1].axhline(y=lstm_ae.threshold, color='r', linestyle='--', label='Threshold')
axes[0, 1].scatter(np.where(anomalies)[0], reconstruction_errors[anomalies],
                  color='red', alpha=0.6, s=20, label='Anomalies')
axes[0, 1].set_title('Reconstruction Errors')
axes[0, 1].set_xlabel('Sequence Index')
axes[0, 1].set_ylabel('MSE')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Detection vs True labels
axes[1, 0].plot(sequence_labels, 'g-', label='True Labels', alpha=0.7)
axes[1, 0].scatter(np.where(anomalies)[0],
                  [3] * sum(anomalies),
                  color='red', alpha=0.5, s=10, label='LSTM-AE Detections')
axes[1, 0].set_title('Anomaly Detection Timeline')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('State')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Confusion Matrix
binary_labels = (sequence_labels > 0).astype(int)
cm = confusion_matrix(binary_labels, anomalies)
sns.heatmap(cm, annot=True, fmt='d', ax=axes[1, 1], cmap='Purples')
axes[1, 1].set_title('LSTM-AE Confusion Matrix')
axes[1, 1].set_xlabel('Predicted')
axes[1, 1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

# 성능 리포트
print("\n📊 LSTM Autoencoder Performance:")
print(classification_report(binary_labels, anomalies,
                           target_names=['Normal', 'Anomaly']))

## 5. 실시간 스트리밍 파이프라인

### 5.1 알람 정책 및 피로도 관리

In [ ]:
class AlarmManagementSystem:
    """알람 관리 시스템 - 알람 피로도 방지"""

    def __init__(self):
        self.alarm_history = []
        self.alarm_policies = {
            'critical': {
                'threshold': 0.9,
                'min_interval': 60,  # seconds
                'max_frequency': 5,  # per hour
                'actions': ['email', 'sms', 'stop_line']
            },
            'high': {
                'threshold': 0.7,
                'min_interval': 300,
                'max_frequency': 10,
                'actions': ['email', 'dashboard']
            },
            'medium': {
                'threshold': 0.5,
                'min_interval': 600,
                'max_frequency': 20,
                'actions': ['dashboard']
            },
            'low': {
                'threshold': 0.3,
                'min_interval': 3600,
                'max_frequency': 50,
                'actions': ['log']
            }
        }

    def evaluate_alarm(self, anomaly_score, timestamp, equipment_id):
        """알람 평가 및 필터링"""
        # 알람 레벨 결정
        alarm_level = self._determine_alarm_level(anomaly_score)

        if alarm_level is None:
            return None

        # 알람 피로도 체크
        if self._check_alarm_fatigue(equipment_id, alarm_level, timestamp):
            return None

        # 알람 생성
        alarm = {
            'timestamp': timestamp,
            'equipment_id': equipment_id,
            'level': alarm_level,
            'score': anomaly_score,
            'actions': self.alarm_policies[alarm_level]['actions']
        }

        self.alarm_history.append(alarm)

        return alarm

    def _determine_alarm_level(self, score):
        """알람 레벨 결정"""
        for level in ['critical', 'high', 'medium', 'low']:
            if score >= self.alarm_policies[level]['threshold']:
                return level
        return None

    def _check_alarm_fatigue(self, equipment_id, level, timestamp):
        """알람 피로도 체크"""
        policy = self.alarm_policies[level]

        # 최근 알람 확인
        recent_alarms = [
            a for a in self.alarm_history
            if a['equipment_id'] == equipment_id and
            a['level'] == level and
            (timestamp - a['timestamp']).total_seconds() < 3600
        ]

        # 빈도 체크
        if len(recent_alarms) >= policy['max_frequency']:
            return True

        # 최소 간격 체크
        if recent_alarms:
            last_alarm_time = recent_alarms[-1]['timestamp']
            if (timestamp - last_alarm_time).total_seconds() < policy['min_interval']:
                return True

        return False

    def get_alarm_statistics(self):
        """알람 통계"""
        if not self.alarm_history:
            return {}

        df = pd.DataFrame(self.alarm_history)

        stats = {
            'total_alarms': len(df),
            'by_level': df['level'].value_counts().to_dict(),
            'by_equipment': df['equipment_id'].value_counts().to_dict(),
            'suppressed_count': 0  # 실제로는 별도로 추적 필요
        }

        return stats

# 실시간 스트리밍 시뮬레이션
class StreamingPipeline:
    """실시간 예지보전 파이프라인"""

    def __init__(self, model, alarm_system):
        self.model = model  # 이상 탐지 모델
        self.alarm_system = alarm_system
        self.buffer = []  # 데이터 버퍼
        self.buffer_size = 100
        self.results = []

    def process_stream(self, data_point, timestamp, equipment_id):
        """실시간 데이터 처리"""
        # 버퍼에 추가
        self.buffer.append(data_point)

        # 버퍼 크기 유지
        if len(self.buffer) > self.buffer_size:
            self.buffer.pop(0)

        # 충분한 데이터가 모이면 분석
        if len(self.buffer) >= 50:
            # 특징 추출
            features = self._extract_features(self.buffer[-50:])

            # 이상 탐지
            anomaly_score = self._detect_anomaly(features)

            # 알람 평가
            alarm = self.alarm_system.evaluate_alarm(
                anomaly_score, timestamp, equipment_id
            )

            # 결과 저장
            result = {
                'timestamp': timestamp,
                'equipment_id': equipment_id,
                'anomaly_score': anomaly_score,
                'alarm': alarm
            }

            self.results.append(result)

            return result

        return None

    def _extract_features(self, window_data):
        """윈도우 데이터에서 특징 추출"""
        # 간단한 통계 특징
        features = [
            np.mean(window_data),
            np.std(window_data),
            np.max(window_data),
            np.min(window_data),
            kurtosis(window_data)
        ]
        return np.array(features)

    def _detect_anomaly(self, features):
        """이상 점수 계산"""
        # 모델 예측 (시뮬레이션)
        # 실제로는 학습된 모델 사용
        anomaly_score = np.random.beta(2, 5)  # 0~1 사이 값

        # 특징값이 비정상적이면 점수 증가
        if features[4] > 3:  # High kurtosis
            anomaly_score = min(1.0, anomaly_score + 0.3)

        return anomaly_score

# 스트리밍 파이프라인 시뮬레이션
alarm_system = AlarmManagementSystem()
pipeline = StreamingPipeline(if_model, alarm_system)

# 실시간 데이터 스트림 시뮬레이션
print("🔄 실시간 스트리밍 파이프라인 시뮬레이션...\n")

stream_results = []
equipment_ids = ['PUMP-001', 'MOTOR-002', 'BEARING-003']

for i in range(200):
    # 시뮬레이션 데이터 생성
    timestamp = datetime.now() + timedelta(minutes=i)
    equipment_id = np.random.choice(equipment_ids)

    # 시간에 따라 점진적으로 열화
    if i < 50:
        data_point = np.random.normal(0, 1)
    elif i < 100:
        data_point = np.random.normal(0.5, 1.5)
    elif i < 150:
        data_point = np.random.normal(1, 2)
    else:
        data_point = np.random.normal(2, 3)

    # 스트림 처리
    result = pipeline.process_stream(data_point, timestamp, equipment_id)

    if result:
        stream_results.append(result)

        # 알람 발생 시 출력
        if result['alarm']:
            print(f"⚠️ ALARM: {result['alarm']['level'].upper()}")
            print(f"   Equipment: {result['alarm']['equipment_id']}")
            print(f"   Score: {result['alarm']['score']:.3f}")
            print(f"   Actions: {', '.join(result['alarm']['actions'])}")
            print()

# 결과 분석
stream_df = pd.DataFrame(stream_results)

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Real-time Streaming Pipeline Results', fontsize=16, fontweight='bold')

# Anomaly scores over time
axes[0, 0].plot(stream_df['timestamp'], stream_df['anomaly_score'])
axes[0, 0].set_title('Anomaly Scores Over Time')
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Anomaly Score')
axes[0, 0].grid(True, alpha=0.3)

# Alarm triggers
alarm_df = stream_df[stream_df['alarm'].notna()]
if not alarm_df.empty:
    alarm_levels = [a['level'] if a else None for a in alarm_df['alarm']]
    level_colors = {'critical': 'red', 'high': 'orange', 'medium': 'yellow', 'low': 'green'}
    colors = [level_colors.get(level, 'gray') for level in alarm_levels]

    axes[0, 1].scatter(alarm_df['timestamp'], alarm_df['anomaly_score'],
                      c=colors, s=50, alpha=0.7)
    axes[0, 1].set_title('Alarm Triggers')
    axes[0, 1].set_xlabel('Time')
    axes[0, 1].set_ylabel('Anomaly Score')
    axes[0, 1].grid(True, alpha=0.3)

# Alarm statistics
alarm_stats = alarm_system.get_alarm_statistics()
if alarm_stats and 'by_level' in alarm_stats:
    levels = list(alarm_stats['by_level'].keys())
    counts = list(alarm_stats['by_level'].values())
    axes[1, 0].bar(levels, counts, color=['red', 'orange', 'yellow', 'green'])
    axes[1, 0].set_title('Alarms by Level')
    axes[1, 0].set_xlabel('Alarm Level')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].grid(True, alpha=0.3)

# Equipment-wise alarms
if alarm_stats and 'by_equipment' in alarm_stats:
    equipment = list(alarm_stats['by_equipment'].keys())
    counts = list(alarm_stats['by_equipment'].values())
    axes[1, 1].pie(counts, labels=equipment, autopct='%1.1f%%')
    axes[1, 1].set_title('Alarms by Equipment')

plt.tight_layout()
plt.show()

# 최종 통계
print("\n📊 스트리밍 파이프라인 최종 통계:")
print(f"   처리된 데이터 포인트: 200")
print(f"   분석된 윈도우: {len(stream_results)}")
if alarm_stats:
    print(f"   발생한 알람: {alarm_stats.get('total_alarms', 0)}")
    print(f"   알람 분포: {alarm_stats.get('by_level', {})}")

## 6. 실습 과제 및 Best Practices

### 6.1 예지보전 구현 체크리스트

In [ ]:
# 예지보전 구현 체크리스트 및 템플릿
predictive_maintenance_checklist = {
    '데이터 수집': {
        'tasks': [
            '센서 종류 및 사양 정의 (진동, 온도, 전류, 음향 등)',
            '샘플링 주파수 설정 (Nyquist 정리 고려)',
            '데이터 저장 인프라 구축',
            '정상/고장 라벨 데이터 확보'
        ],
        'considerations': [
            '진동 센서: 최소 10kHz 샘플링',
            '온도 센서: 1Hz 이상',
            '최소 3개월 이상 데이터 수집 권장'
        ]
    },
    '특징 추출': {
        'time_domain': [
            'RMS (Root Mean Square)',
            'Kurtosis (충격 신호 감지)',
            'Crest Factor',
            'Peak-to-Peak'
        ],
        'frequency_domain': [
            'FFT Spectrum',
            'Envelope Analysis',
            'Order Analysis',
            'Spectral Entropy'
        ],
        'time_frequency': [
            'Wavelet Transform',
            'STFT (Short-Time Fourier Transform)',
            'Hilbert-Huang Transform'
        ]
    },
    '모델 선택': {
        'unsupervised': [
            'Isolation Forest: 빠르고 효과적',
            'One-Class SVM: 비선형 패턴',
            'Autoencoder: 복잡한 패턴'
        ],
        'supervised': [
            'Random Forest: 해석 가능',
            'XGBoost: 높은 성능',
            'LSTM: 시계열 패턴'
        ]
    },
    '배포 및 운영': {
        'deployment': [
            'Edge Computing 고려',
            'API 서버 구축',
            '실시간 처리 파이프라인',
            '모니터링 대시보드'
        ],
        'maintenance': [
            '모델 재학습 주기 설정',
            '드리프트 감지',
            '성능 모니터링',
            'A/B 테스트'
        ]
    }
}

# RUL (Remaining Useful Life) 예측 템플릿
rul_template = {
    'equipment_info': {
        'equipment_id': '',
        'type': '',  # pump, motor, bearing, etc.
        'installation_date': '',
        'operating_hours': 0,
        'maintenance_history': []
    },
    'sensor_config': {
        'vibration': {'sampling_rate': 10000, 'channels': 3},
        'temperature': {'sampling_rate': 1, 'channels': 1},
        'current': {'sampling_rate': 100, 'channels': 3}
    },
    'model_config': {
        'type': 'LSTM',
        'input_features': [],
        'prediction_horizon': 30,  # days
        'confidence_interval': 0.95
    },
    'alert_config': {
        'rul_thresholds': {
            'critical': 7,   # days
            'warning': 30,   # days
            'notice': 90     # days
        }
    }
}

# 체크리스트 출력
print("✅ 예지보전 구현 체크리스트\n")
for category, items in predictive_maintenance_checklist.items():
    print(f"📌 {category}:")
    for key, values in items.items():
        print(f"\n  [{key.upper()}]")
        for value in values:
            print(f"    □ {value}")
    print()

# JSON 파일로 저장
import json

with open('predictive_maintenance_template.json', 'w', encoding='utf-8') as f:
    json.dump({
        'checklist': predictive_maintenance_checklist,
        'rul_template': rul_template
    }, f, indent=2, ensure_ascii=False)

print("\n💾 템플릿이 'predictive_maintenance_template.json'으로 저장되었습니다.")

## 7. Session 3 핵심 요약

### 이번 세션에서 학습한 내용:

1. **시계열 특징 추출**
   - 시간 도메인: RMS, Kurtosis, Crest Factor
   - 주파수 도메인: FFT, Spectral Features
   - Envelope Analysis for bearing faults

2. **이상 탐지 모델 비교**
   - Isolation Forest: 빠르고 효율적
   - One-Class SVM: 비선형 경계
   - LSTM Autoencoder: 시계열 패턴 학습

3. **알람 관리 시스템**
   - 알람 피로도 방지 전략
   - 다단계 알람 정책
   - 실시간 스트리밍 처리

4. **실전 고려사항**
   - 센서 사양 및 샘플링 주파수
   - Edge vs Cloud 처리
   - 모델 드리프트 관리

### Best Practices:

1. **데이터 품질**
   - 충분한 정상 데이터 확보 (최소 3개월)
   - 다양한 운전 조건 포함
   - 센서 캘리브레이션 정기 수행

2. **모델 선택**
   - 초기: 단순 통계 기반 방법
   - 중기: Isolation Forest, OCSVM
   - 고도화: Deep Learning (데이터 충분 시)

3. **운영 최적화**
   - 점진적 임계값 조정
   - 현장 피드백 반영
   - 정기적 재학습

### ROI 극대화 전략:
- Critical Equipment 우선 적용
- 고장 비용이 높은 설비 집중
- 단계적 확대 (Pilot → Scale-up)
- 운영 인력 교육 병행

In [ ]:
print("🎯 제조 AI 6시간 과정 완료!\n")
print("학습한 내용:")
print("✅ Session 1: 제조 데이터 아키텍처 & 과제 정의")
print("✅ Session 2: 비전 불량검사 고도화")
print("✅ Session 3: 예지보전(시계열) 심화\n")
print("💡 실무 적용 팁:")
print("1. Small Start, Quick Win 전략")
print("2. 현장 작업자와 긴밀한 협업")
print("3. 지속적 모니터링과 개선")
print("4. ROI 중심 의사결정\n")
print("📧 추가 문의: imguru.co.kr")
print("\n감사합니다! 🙏")